In [1]:
import ee
import geemap

In [24]:
js_snippet = """
  
"""
geemap.js_snippet_to_py(js_snippet)

In [27]:
import ee
import geemap

Map = geemap.Map()

table = ee.FeatureCollection("projects/ee-fernandotentor/assets/Agreed/Outwoods_Warren")
farms = ee.ImageCollection("projects/ee-fernandotentor/assets/Agreed/Farms")
#########/  S2

stardate = '2017-04-01'
enddate = '2022-01-01'

start = ee.Date(stardate)
end = ee.Date(enddate)
timedelta = end.difference(start, 'day')
aggregation = 1
timeunit = 'month'; #'day' or 'month'
scale = 100

imageCollection = ee.ImageCollection("COPERNICUS/S2_SR")

########  function Sentinel 2 NDVI

def add_ndvi (image):
  ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
  return image.addBands(ndvi)

########  function Sentinel 2 NDMI
def add_ndmi (image):
  ndvi = image.normalizedDifference(['B8','B11']).rename('ndmi')
  return image.addBands(ndvi)

######/ Cloud mask

def cloudmask (image):
  mask = image.select('SCL').gte(8).And(image.select('SCL').neq(11))
  return image.mask(mask.neq(1))

def get_data(table):
  S2 = imageCollection \
    .filterDate('2015-06-01', enddate) \
    .filterBounds(table) \
    .map(cloudmask) \
    .map(add_ndvi) \
    .map(add_ndmi) \
    .select(['ndvi','ndmi'])

  # Aggregation

  def step (count):
    return start.advance(count, timeunit)

  def filter (date):
    images_g = S2.filterDate(date,ee.Date(date).advance(aggregation,timeunit))
    return images_g.mean().set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))

  S2_list = ee.List.sequence(0, timedelta.int(),aggregation) \
    .map(step) \
    .map(filter)

  # Check missing values
  def check (image):
    return ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image))

  S2_list_clean = S2_list.map(check, True)

  return S2_list_clean



In [28]:
S2_list_clean = get_data(table)